# Introduction to LLMs and Agents

Welcome to our workshop! In this session, we'll explore how to build AI-powered applications using **LangChain**, a popular framework for developing applications with Large Language Models (LLMs). We'll start with a simple chatbot and then enhance it with a multi-agent framework.

## Setting Up Our Environment

First, we need to set up our environment. We'll use OpenAI's models, so we need an API key. You can define your `OPENAI_API_KEY` in the `.env` file.

The code retrieve the key and sets some global configurations:
- `LLM_MODEL`: The specific model we'll use
- `LLM_TEMPERATURE`: Controls randomness in responses (0 means very deterministic)

In [1]:
import os

In [2]:
if not os.environ.get("OPENAI_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY environment variable")

LLM_MODEL = "gpt-4o-mini"
LLM_TEMPERATURE = 0.9

In [3]:
"""A simple edit of the original yfinance tool to make it less restrictive."""

from typing import Iterable, Optional, Type

from langchain_core.callbacks import CallbackManagerForToolRun
from langchain_core.documents import Document
from langchain_core.tools import BaseTool
from pydantic import BaseModel, Field
from requests.exceptions import HTTPError, ReadTimeout
from urllib3.exceptions import ConnectionError

from langchain_community.document_loaders.web_base import WebBaseLoader


class YahooFinanceNewsInput(BaseModel):
    """Input for the YahooFinanceNews tool."""

    query: str = Field(description="company ticker query to look up")


class YahooFinanceNewsTool(BaseTool):
    """Tool that searches financial news on Yahoo Finance."""

    name: str = "yahoo_finance_news"
    description: str = (
        "Useful for when you need to find financial news "
        "about a public company. "
        "Input should be a company ticker. "
        "For example, AAPL for Apple, MSFT for Microsoft."
    )
    top_k: int = 10
    """The number of results to return."""

    args_schema: Type[BaseModel] = YahooFinanceNewsInput

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """
        Use the Yahoo Finance News tool.

        Args:
            query: Company ticker symbol (e.g., 'AAPL' for Apple).
            run_manager: Optional callback manager.

        Returns:
            str: Formatted news results or error message.
        """
        try:
            import yfinance
        except ImportError:
            raise ImportError(
                "Could not import yfinance python package. "
                "Please install it with `pip install yfinance`."
            )
        company = yfinance.Ticker(query)

        try:
            if company.isin is None:
                return f"Company ticker {query} not found."
        except (HTTPError, ReadTimeout, ConnectionError):
            return f"Company ticker {query} not found."

        links = []

        try:
            links = [
                n["content"]["canonicalUrl"]["url"]
                for n in company.news
                if n["content"]["contentType"] == "STORY"
            ]
            # print(links)

        except (HTTPError, ReadTimeout, ConnectionError):
            if not links:
                return f"No news found for company that searched with {query} ticker."
        if not links:
            return f"No news found for company that searched with {query} ticker."
        loader = WebBaseLoader(web_paths=links)
        docs = loader.load()
        result = self._format_results(docs, query)
        if not result:
            return f"No news found for company that searched with {query} ticker."
        return result

    @staticmethod
    def _format_results(docs: Iterable[Document], query: str) -> str:
        doc_strings = [
            "\n".join([doc.metadata["title"], doc.metadata.get("description", "")])
            for doc in docs
            # if query in doc.metadata.get("description", "")
            # or query in doc.metadata["title"]
        ]
        return "\n\n".join(doc_strings)


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
# from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
# from src.lauzhack_agentic import YahooFinanceNewsTool

tools = [YahooFinanceNewsTool()]

In [5]:
tool = YahooFinanceNewsTool()
tool.args

{'query': {'description': 'company ticker query to look up',
  'title': 'Query',
  'type': 'string'}}

## Building a Simple ChatBot

Let's start with creating a basic chatbot using **LangChain**. We'll use:
- `ChatOpenAI`: The interface to OpenAI's chat models
- `SystemMessage`: Defines the bot's behavior and role
- `HumanMessage`: Represents user input

Our chatbot will act as a Financial Analyst. We'll create it by:
1. Instantiating the model
2. Defining a system prompt that sets the bot's role
3. Sending a user query and getting a response with `.invoke()`

This demonstrates the basic pattern of LLM interactions: prompt → response.

In [6]:
from IPython.display import Markdown
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

In [7]:
# Create a ChatOpenAI instance with the LLM model and temperature
base_model = ChatOpenAI(model=LLM_MODEL, temperature=LLM_TEMPERATURE)

In [8]:
BASE_PROMPT = """
You are a Financial Analyst. Do your best to help the client with their request based on your expertise. Give a clear and succint financial strategy with precise numbers and allocations.
"""

In [9]:
# Request from the client
request = "I'm 25 year old and have $1,000 saved. which US stocks should I invest into?"

# Message list for the base model
messages = [
    SystemMessage(BASE_PROMPT),
    HumanMessage(request),
]

# Invoke the model with the messages
response = base_model.invoke(messages)

In [10]:
Markdown(response.content)

As a 25-year-old with $1,000 to invest, it's important to focus on a diversified strategy that balances growth potential with risk management. Here’s a suggested allocation and strategy:

### Investment Strategy:

1. **Diversification**: Instead of allocating all your funds to individual stocks, consider investing in a mix of ETFs (Exchange-Traded Funds) and a few individual stocks. This way, you can spread risk across different sectors.

2. **Long-term Growth Focus**: Given your age, you can take on more risk for potentially higher returns. Look for stocks and ETFs that have a history of growth and solid fundamentals.

### Suggested Allocation:

- **60% in ETFs**: $600
    - **S&P 500 ETF (e.g., SPY, VOO)**: $300
      - This provides exposure to 500 of the largest U.S. companies, offering diversification and market returns.
    - **Total Stock Market ETF (e.g., VTI)**: $300
      - This covers a broader market including small, mid, and large-cap stocks, enhancing your diversification.

- **30% in Individual Stocks**: $300
    - **Technology Stock (e.g., Apple - AAPL)**: $150
      - Solid track record of growth and innovation.
    - **Consumer Discretionary (e.g., Amazon - AMZN)**: $150
      - Strong position in e-commerce and technology services; growth potential remains significant.

- **10% in a High-Yield Savings Account**: $100
    - Keep this as an emergency fund or for future investment opportunities. It can provide a buffer and keep your money liquid.

### Additional Recommendations:

- **Reinvest Dividends**: Opt for a dividend reinvestment plan (DRIP) for stocks, which allows dividends to be automatically reinvested to purchase more shares.
  
- **Regular Contributions**: Try to contribute additional savings regularly (e.g., $100/month) to enhance your investment positions over time. 

- **Educate Yourself**: Continue to learn about investing, financial markets, and personal finance to make informed decisions in the future.

This strategy positions you for growth while maintaining a level of diversification to mitigate risk. Always consider your risk tolerance and investment horizon before proceeding.

## Simple Agent with Yahoo Finance News

In [11]:
from langchain_core.messages import HumanMessage, ToolMessage

In [12]:
# Create a list of tools and a dictionnary of tool functions by name
tools = [YahooFinanceNewsTool()]
tools_by_name = {tool.name: tool for tool in tools}


In [13]:
user_question = (
    # "I'm 25 year old and have $1,000 saved. which US stocks should I invest into?"
    "How does Microsoft feels today comparing with Nvidia?"
)
FINANCE_TOOL_PROMPT = """
You are a Financial Analyst. The client will ask you a question, and you will give them financial advice.
Then based on the stocks you advise, use the Yahoo Finance tool to get news if it's worth buying currently.
Give clear investment advice at the end. Do not assess risk.
"""

task_str = f"User question: {user_question}"

fa_model = base_model.bind_tools(tools)

In [14]:
# Message list for the financial assistant model
messages = [
    SystemMessage(FINANCE_TOOL_PROMPT),
    HumanMessage(task_str),
]

# Invoke the financial assistant model with the messages
fa_output = fa_model.invoke(messages)

# If the financial assistant model made tool calls, invoke the tool
if fa_output.tool_calls:
    news_list = []
    id_list = []
    for tool_call in fa_output.tool_calls:
        tool = tools_by_name[tool_call["name"]]
        news = tool.invoke(tool_call["args"])

        display(Markdown(f"**Yahoo Finance news**: {news}"))

        news_list.append(news)
        id_list.append(tool_call["id"])

    # Combine the retrieved documents into a single string
    news_str = news
    # Message list with the retrieved documents for the base model
    messages = [
        SystemMessage(FINANCE_TOOL_PROMPT),
        HumanMessage(task_str),
        fa_output,
        *[
            ToolMessage(news_str, tool_call_id=tool_call_id)
            for (news_str, tool_call_id) in zip(news_list, id_list)
        ],
    ]

    # Invoke the base model with the messages
    fa_output_final = fa_model.invoke(messages)


Markdown(fa_output_final.content)

**Yahoo Finance news**: Yahoo is part of the Yahoo family of brands.


OpenAI and Microsoft renegotiate partnership for IPO and funding
OpenAI and Microsoft are engaged in negotiations to redefine their multibillion-dollar partnership, reported Financial Times, citing sources.

Yahoo is part of the Yahoo family of brands.


Yahoo is part of the Yahoo family of brands.


Yahoo is part of the Yahoo family of brands.


Yahoo is part of the Yahoo family of brands.


Yahoo is part of the Yahoo family of brands.


Yahoo is part of the Yahoo family of brands.


Yahoo is part of the Yahoo family of brands.


**Yahoo Finance news**: Yahoo is part of the Yahoo family of brands.


Yahoo is part of the Yahoo family of brands.


What Lessons Can Investors Learn From Cathie Wood's Ark Invest's Aggressive Investment Philosophy? | The Motley Fool


barrons.com


Yahoo is part of the Yahoo family of brands.


Yahoo is part of the Yahoo family of brands.


Yahoo is part of the Yahoo family of brands.


Yahoo is part of the Yahoo family of brands.


Philips and NVIDIA partner to leverage AI for MR imaging
Philips has entered into a partnership with NVIDIA to leverage advanced AI technology for magnetic resonance imaging (MRI).

Here’s a comparison of Microsoft and Nvidia based on the latest news:

### **Microsoft (MSFT)**
- **Recent News**: Microsoft is currently involved in negotiations to redefine its multibillion-dollar partnership with OpenAI, as indicated by sources from the Financial Times. These negotiations could significantly impact their collaborative efforts and future projects, especially in AI.

### **Nvidia (NVDA)**
- **Recent News**: Nvidia has entered a partnership with Philips to leverage its advanced AI technology for magnetic resonance imaging (MRI). This partnership highlights Nvidia’s ongoing commitment to integrating AI into various sectors, particularly healthcare.

### **Investment Outlook**
Both companies are involved in transformational technologies, with Microsoft focusing on AI partnerships and Nvidia expanding its applications of AI into healthcare. Given the current market emphasis on AI advancements, both stocks could be seen as attractive depending on your investment strategy.

### **Final Investment Advice**
- **Microsoft (MSFT)**: Given the potential for growth from its partnership with OpenAI, it could be a good time to consider investing in MSFT.
- **Nvidia (NVDA)**: With its ongoing commitment to AI applications in healthcare, investing in NVDA could also be appealing.

If you are looking for stocks that have strong growth potential in the AI sector, both Microsoft and Nvidia are solid choices.

## Agentic system

We will now create a simple agentic system consisting of three agents using LangChain.
This system will help us perform a more elaborate financial analysis by including:

- Client Interface Agent: Rephrases the user’s prompt to improve the quality of the financial analyst’s response.

- Financial Analyst: Similar to the first part; provides financial advice based on the refined prompt.

- Risk Advisor: Assesses the risk associated with the advice given by the financial analyst.

In [15]:
from typing import Literal

from langchain_core.messages import SystemMessage
from langgraph.constants import Send
from langgraph.graph import END
from langgraph.types import Command

## LangGraph Workflow and State

**LangGraph** helps us manage communication between our agents efficiently by defining a `State` class that will convey information from a node to the next during execution.

We keep our state simple by including only two attributes, but it's possible to include more:
- Messages: The ongoing conversation chain
- Analyses: Research findings from our agents

We use Python's dataclasses with special annotations (`Annotated`) to define how the state attributes should be updated throughout the workflow.

In [16]:
import operator
from dataclasses import dataclass, field
from typing import Annotated

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

In [17]:
@dataclass(kw_only=True)
class State:
    """Graph state for the financial analysis workflow."""

    messages: Annotated[list[BaseMessage], add_messages] = field(default_factory=list)
    yahoo_finance_news: Annotated[list[BaseMessage], add_messages] = field(
        default_factory=list
    )
    risk_analysis: Annotated[list[BaseMessage], add_messages] = field(
        default_factory=list
    )

## Orchestrator

In [18]:
ORCHESTRATOR_PROMPT = """
You are a Client Interface Agent (CIA) in a financial analysis system. You have multiple Research Agents with access to Bloomberg Financial News under your supervision.

Given a client request, provide a concise, polite and professional response regarding the feasibility of the request and the approach that will be taken to address it.

If the user's request is addressable, create a short list of highly specific research topics that the Research Agents will investigate to fulfill the client's request.
"""
# Create the orchestrator model model from the base model with tool binding
bra_model = base_model.bind_tools(tools)

In [19]:
# Define the CIA orchestrator with the next node options
def orchestrator_node(state: State) -> Command[Literal["worker", END]]:
    """Orchestrator that generates a plan for the financial analysis."""
    display(Markdown(f"**Client request received**: {state.messages[-1].content}"))

    # Message list for the orchestrator model
    messages = [
        SystemMessage(ORCHESTRATOR_PROMPT),
        *state.messages,
    ]

    # Invoke the CIA model
    orchestrator_output = base_model.invoke(messages)

    display(Markdown(f"**CIA Response:** {orchestrator_output.response}"))

    return Command(
        # Update the state messages with the CIA response
        update={"messages": orchestrator_output.response},
        # Go to worker nodes if the request is in scope, otherwise end the workflow
        goto=[Send("worker", task) for task in orchestrator_output.research_tasks]
        if orchestrator_output.in_scope
        else END,
    )

## Worker Nodes

### Financial analyst Node

In [20]:
from langchain_core.messages import HumanMessage, ToolMessage

In [21]:
FINANCE_TOOL_PROMPT = """
You are a Financial Analyst. The client will ask you a question, and you will give them financial advice.
Then based on the stocks you advise, use the Yahoo Finance tool to get news if it's worth buying currently.
Give clear investment advice at the end. Do not assess risk.
"""

# Create the financial assistant model from the base model with tool binding
fa_model = base_model.bind_tools(tools)

In [33]:
# Define the worker node and the next node options
def financial_analyst_node(state: State) -> Command[Literal["synthesizer"]]:
    """Given a user question, get financial advice from Yahoo Finance news."""
    display(Markdown("**Giving financial advice without risk information**"))
    # Message list for the financial assistant model
    messages = [
        SystemMessage(FINANCE_TOOL_PROMPT),
        *state.messages,
    ]

    # Invoke the financial assistant model with the messages
    fa_output = fa_model.invoke(messages)

    # If the financial assistant model made tool calls, invoke the tool
    if fa_output.tool_calls:
        news_list = []
        id_list = []
        for tool_call in fa_output.tool_calls:
            tool = tools_by_name[tool_call["name"]]
            news = tool.invoke(tool_call["args"])

            display(Markdown(f"**Yahoo Finance news**: {news}"))

            news_list.append(news)
            id_list.append(tool_call["id"])

        # Message list with the retrieved documents for the base model
        messages = [
            SystemMessage(FINANCE_TOOL_PROMPT),
            HumanMessage(task_str),
            fa_output,
            *[
                ToolMessage(news_str, tool_call_id=tool_call_id)
                for (news_str, tool_call_id) in zip(news_list, id_list)
            ],
        ]

        # Invoke the base model with the messages
        fa_output = fa_model.invoke(messages)

    # Update the state analyses with the financial analyst output content and go to the synthesizer node
    # NOTE: To update `analyses` you should return a list
    return Command(
        update={"yahoo_finance_news": [fa_output.content]},
        goto="synthesizer",
    )

### Risk analyst Node

In [34]:
from langchain_core.messages import AIMessage

In [35]:
RISK_ANALYST_PROMPT = """
You are a Risk Advisor.
Evaluate the analyst's advice from a risk perspective. Offer any cautions and ways to reduce risk, don't repeat the analyst's advice.
"""

In [36]:
# Define the synthesizer node and the next node options
def risk_analyst_node(state: State) -> Command[Literal[END]]:
    """Given the user prompt perform risk analysis."""
    display(Markdown("**Performing risk analysis.**"))

    # Access the previous responses
    finance_analysis = state.yahoo_finance_news

    # Combine the previous messages into a single string
    complete_analyses = "The financial advisor's analysis:" + "\n\n---\n\n".join(
        finance_analysis
    )

    # Message list for the risk analyst
    messages = [
        SystemMessage(RISK_ANALYST_PROMPT),
        *state.messages,
        AIMessage(complete_analyses),
    ]

    # Invoke the base model with the messages
    risk_analyst_output = base_model.invoke(messages)

    # Update the state messages with the risk analyst's output content and go to the synthesizer node
    return Command(
        update={"risk_analysis": [risk_analyst_output.content]},
        goto="synthesizer",
    )

### Synthesiser node

In [37]:
from langchain_core.messages import AIMessage

In [38]:
SYNTHESISER_PROMPT = """
Your are a financial analyst, take the general financial analyst's insights and the risk analyst's insights and combine them to generate comprehensive financial advise for the user.
"""

In [ ]:
# Define the synthesizer node and the next node options
def synthesizer_node(state: State) -> Command[Literal[END]]:
    """Synthesize full report from research analyses."""
    display(Markdown("**Synthesizing messages from the two analysts.**"))

    # Access the previous responses
    finance_analysis = state.yahoo_finance_news

    risk_analysis = state.risk_analysis

    # Combine the research analyses into a single string
    financial_analysis_str = "risk analysis: " + "\n\n---\n\n".join(
        [item.content for item in finance_analysis]
    )
    risk_analysis_str = "risk analysis: " + "\n\n---\n\n".join(
        [item.content for item in risk_analysis]
    )

    # Message list for the RSA model
    messages = [
        SystemMessage(SYNTHESISER_PROMPT),
        AIMessage(financial_analysis_str),
        AIMessage(risk_analysis_str),
    ]

    # Invoke the base model with the messages
    synth_output = base_model.invoke(messages)

    # Update the state messages with the RSA output content and end the workflow
    return Command(
        update={"messages": synth_output},
        goto=END,
    )

## Building the Workflow Graph

Now that our nodes and communication flow are defined, we can build the graph!

In [60]:
from IPython.display import Image
from langgraph.graph import StateGraph

In [61]:
# Create a state graph builder
graph_builder = StateGraph(State)

# Define the entry point
graph_builder.set_entry_point("financial_analyst")
graph_builder.set_entry_point("risk_analyst")
# Add the nodes
# graph_builder.add_node("orchestrator", orchestrator_node)
graph_builder.add_node("financial_analyst", financial_analyst_node)
graph_builder.add_node("risk_analyst", risk_analyst_node)
graph_builder.add_node("synthesizer", synthesizer_node)

# The edges are defined by the commands !

# Compile the workflow
app = graph_builder.compile()

Let's visualize our Financial Analyst graph. Note that because the number of `"worker"` nodes is generated dynamically, it shows up as a single node in the image.

In [62]:
# display(Image(app.get_graph().draw_mermaid_png()))

## Running the Workflow

Now that our workflow is built, let's test it! Once again, we can run it with `.invoke()`.

In [63]:
request = "I want to invest in the technology sector. Can you please define an investment strategy?"

# Invoke the workflow with the client request
final_state = app.invoke({"messages": request})

**Performing risk analysis.**

**Giving financial advice without risk information**

**Yahoo Finance news**: Yahoo is part of the Yahoo family of brands.


Yahoo is part of the Yahoo family of brands.


Yahoo is part of the Yahoo family of brands.


Yahoo is part of the Yahoo family of brands.


wsj.com


Yahoo is part of the Yahoo family of brands.


Yahoo is part of the Yahoo family of brands.


Tesla Stock vs. Apple Stock: The Best buy Right Now, According to Wall Street | The Motley Fool


barrons.com


**Yahoo Finance news**: Yahoo is part of the Yahoo family of brands.


OpenAI and Microsoft renegotiate partnership for IPO and funding
OpenAI and Microsoft are engaged in negotiations to redefine their multibillion-dollar partnership, reported Financial Times, citing sources.

Yahoo is part of the Yahoo family of brands.


Yahoo is part of the Yahoo family of brands.


Yahoo is part of the Yahoo family of brands.


Yahoo is part of the Yahoo family of brands.


Yahoo is part of the Yahoo family of brands.


Yahoo is part of the Yahoo family of brands.


Yahoo is part of the Yahoo family of brands.


**Yahoo Finance news**: Yahoo is part of the Yahoo family of brands.


Yahoo is part of the Yahoo family of brands.


What Lessons Can Investors Learn From Cathie Wood's Ark Invest's Aggressive Investment Philosophy? | The Motley Fool


barrons.com


Yahoo is part of the Yahoo family of brands.


Yahoo is part of the Yahoo family of brands.


Yahoo is part of the Yahoo family of brands.


Yahoo is part of the Yahoo family of brands.


Philips and NVIDIA partner to leverage AI for MR imaging
Philips has entered into a partnership with NVIDIA to leverage advanced AI technology for magnetic resonance imaging (MRI).

**Synthesizing messages from the two analysts.**

In [67]:
Markdown(final_state["messages"][-1].content)

Based on the insights gathered from both general financial and risk analysis perspectives, here’s a comprehensive financial advisory tailored for you:

### Comprehensive Financial Advisory

#### 1. **Investment Selection**
   - **Consider High-Growth Stocks**: Companies like **Microsoft (MSFT)** are at the forefront of innovative technologies—especially with its partnership with OpenAI—which positions them favorably within the AI sector. Similarly, **NVIDIA (NVDA)** is also leveraging AI technology to push boundaries in healthcare and computing.
   - **Value in Stability**: While **Apple (AAPL)** may not have recent headlines, its strong fundamentals and brand loyalty make it a resilient option in your portfolio. Consider maintaining a stake in Apple for its stability and reliability.

#### 2. **Diversification**
   - **Sector Allocation**: Diversify your portfolio by allocating capital across various technology sectors as well as other industries (e.g., healthcare, consumer goods). This can cushion against sector-specific downturns.
   - **Geographic Spread**: Consider international markets as well to diversify risks and tap into emerging markets that show potential for technology growth.

#### 3. **Risk Management**
   - **Valuation Scrutiny**: Carefully evaluate each investment’s fundamental metrics to avoid overvalued stocks. High-growth tech stocks can appear attractive, but ensure they reflect sustainable growth.
   - **Regulatory Awareness**: Keep an eye on legislative developments that may impact the tech industry, particularly regarding data privacy and anti-competitive practices.

#### 4. **Investment Horizon and Strategy**
   - **Long-Term Perspective**: Aim for a long-term investment strategy given technology’s volatility. Position your investments for growth over several years rather than short-term gains to mitigate risks from market fluctuations.
   - **Regular Reviews**: Conduct quarterly reviews of your portfolio to reassess your investment thesis based on market changes, company performance, and economic indicators.

#### 5. **Utilizing Tools and Orders**
   - **Stop-Loss Orders**: Utilize stop-loss orders to protect against significant downturns. This can safeguard your portfolio from unexpected market drops.
   - **Index Funds/ETFs**: If you’re less inclined to pick individual stocks, consider investing in tech-focused index funds or ETFs. This offers diversification and mitigates individual stock risk.

#### 6. **Stay Informed**
   - **Continue Research**: Ongoing education about market trends, technological advancements, and economic factors is crucial. Subscribe to reputable financial news sources and regularly analyze market reports.

#### 7. **Portfolio Balance**
   - **Cash Allocation**: Maintain a portion of your portfolio in cash or cash-equivalents for liquidity. This can be advantageous during market corrections for buying opportunities.

### Conclusion
In summary, aligning your investment strategy with both growth opportunities and risk management principles can enhance your potential for positive financial outcomes. By staying informed and flexible in your approach, you can effectively navigate the complexities of the technology sector and optimize your portfolio for long-term success.